<a href="https://colab.research.google.com/github/DataEngineering-Amber/Collabs-and-Assignments/blob/main/completed9_18_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Data Modeling and Extraction from APIs

In this notebook, we will learn how to fetch data from a public API, JSONPlaceholder, and store it in an SQLite database. We will also practice SQL queries to manipulate and retrieve data.

### Concepts Covered:
- Fetching data from an API using `requests` library
- Creating SQLite tables
- Inserting data into tables
- Querying the data using SQL commands

## Step 1: Setup the SQLite Database
We will first create the necessary tables that match the JSONPlaceholder data schema.

In [1]:
import sqlite3
import pandas as pd
from IPython.display import display
import uuid

# Create a new SQLite database
conn = sqlite3.connect(str(uuid.uuid4()) + '.db')
cursor = conn.cursor()

# Create tables
create_users_table = '''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    username TEXT,
    email TEXT,
    address TEXT,
    phone INTEGER,
    website TEXT,
    company TEXT
);
'''

create_posts_table = '''
CREATE TABLE IF NOT EXISTS posts (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    body TEXT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''

create_comments_table = '''
CREATE TABLE IF NOT EXISTS comments (
    id INTEGER PRIMARY KEY,
    post_id INTEGER,
    name TEXT,
    email TEXT,
    body TEXT,
    FOREIGN KEY (post_id) REFERENCES posts(id)
);
'''

create_albums_table = '''
CREATE TABLE IF NOT EXISTS albums (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''

create_photos_table = '''
CREATE TABLE IF NOT EXISTS photos (
    id INTEGER PRIMARY KEY,
    album_id INTEGER,
    title TEXT,
    url TEXT,
    thumbnail_url TEXT,
    FOREIGN KEY (album_id) REFERENCES albums(id)
);
'''

create_todos_table = '''
CREATE TABLE IF NOT EXISTS todos (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    completed BOOLEAN,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''

# Execute the SQL commands
cursor.execute(create_users_table)
cursor.execute(create_posts_table)
cursor.execute(create_comments_table)
cursor.execute(create_albums_table)
cursor.execute(create_photos_table)
cursor.execute(create_todos_table)

# Commit the changes
conn.commit()

# Verify the tables were created
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('users',), ('posts',), ('comments',), ('albums',), ('photos',), ('todos',)]

## Step 2: Fetch Data from JSONPlaceholder API
We will use the `requests` library to fetch data from the API for each entity: users, posts, comments, albums, photos, and todos.

In [2]:
import requests


# Define function to fetch data from API
def fetch_data(endpoint):
    url = f'https://jsonplaceholder.typicode.com/{endpoint}'
    response = requests.get(url)
    return response.json()

# Fetch data
users_data = fetch_data('users')
posts_data = fetch_data('posts')
comments_data = fetch_data('comments')
albums_data = fetch_data('albums')
photos_data = fetch_data('photos')
todos_data = fetch_data('todos')

# Print example data
users_data[:2], posts_data[:2], comments_data[:2]

([{'id': 1,
   'name': 'Leanne Graham',
   'username': 'Bret',
   'email': 'Sincere@april.biz',
   'address': {'street': 'Kulas Light',
    'suite': 'Apt. 556',
    'city': 'Gwenborough',
    'zipcode': '92998-3874',
    'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
   'phone': '1-770-736-8031 x56442',
   'website': 'hildegard.org',
   'company': {'name': 'Romaguera-Crona',
    'catchPhrase': 'Multi-layered client-server neural-net',
    'bs': 'harness real-time e-markets'}},
  {'id': 2,
   'name': 'Ervin Howell',
   'username': 'Antonette',
   'email': 'Shanna@melissa.tv',
   'address': {'street': 'Victor Plains',
    'suite': 'Suite 879',
    'city': 'Wisokyburgh',
    'zipcode': '90566-7771',
    'geo': {'lat': '-43.9509', 'lng': '-34.4618'}},
   'phone': '010-692-6593 x09125',
   'website': 'anastasia.net',
   'company': {'name': 'Deckow-Crist',
    'catchPhrase': 'Proactive didactic contingency',
    'bs': 'synergize scalable supply-chains'}}],
 [{'userId': 1,
   'id': 1,
   'tit

## Step 3: Insert Data into SQLite Tables
Now we will insert the fetched data into their corresponding SQLite tables.

In [3]:
# Insert users data into the users table
def insert_users(data):
    for user in data:
        cursor.execute("""
        INSERT INTO users (id, name, username, email, address, phone, website, company)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (user['id'], user['name'], user['username'], user['email'],
              str(user['address']), user['phone'], user['website'], str(user['company'])))
    conn.commit()

# Repeat similar for other tables
insert_users(users_data)
cursor.execute('SELECT * FROM users LIMIT 5;')
cursor.fetchall()

# Insert posts data into posts table
def insert_posts(data):
    for post in data:
        cursor.execute("""
            INSERT INTO posts (id, user_id, title, body)
            VALUES (?, ?, ?, ?)
        """, (post['id'], post['userId'], post['title'], post['body']))
    conn.commit()

insert_posts(posts_data)
print("Posts data inserted successfully.")
cursor.execute('SELECT * FROM posts LIMIT 5;')
cursor.fetchall()

# Insert comments data into comments table
def insert_comments(data):
    for comment in data:
        cursor.execute("""
            INSERT INTO comments (id, post_id, name, email, body)
            VALUES (?, ?, ?, ?, ?)
        """, (comment['id'], comment['postId'], comment['name'], comment['email'], comment['body']))
    conn.commit()

insert_comments(comments_data)
print("Comments data inserted successfully.")

# Insert albums data into albums table
def insert_albums(data):
    for album in data:
        cursor.execute("""
            INSERT INTO albums (id, user_id, title)
            VALUES (?, ?, ?)
        """, (album['id'], album['userId'], album['title']))
    conn.commit()

insert_albums(albums_data)
print("Albums data inserted successfully.")

# Insert photos data into photos table
def insert_photos(data):
    for photo in data:
        cursor.execute("""
            INSERT INTO photos (id, album_id, title, url, thumbnail_url)
            VALUES (?, ?, ?, ?, ?)
        """, (photo['id'], photo['albumId'], photo['title'], photo['url'], photo['thumbnailUrl']))
    conn.commit()

insert_photos(photos_data)
print("Photos data inserted successfully.")

Posts data inserted successfully.
Comments data inserted successfully.
Albums data inserted successfully.
Photos data inserted successfully.


## Step 4: Query Data from SQLite Tables
Now let's run some SQL queries to retrieve the data we inserted.

In [4]:
# Example: Fetch posts made by user with ID 1
cursor.execute("SELECT count(*) FROM posts where user_id = 1")
user_posts = cursor.fetchall()
user_posts

[(10,)]

In [ ]:
# Example: Fetch comments on post with ID 1
cursor.execute("SELECT * FROM comments WHERE post_id = 1;")
post_comments = cursor.fetchall()
post_comments

[(1,
  1,
  'id labore ex et quam laborum',
  'Eliseo@gardner.biz',
  'laudantium enim quasi est quidem magnam voluptate ipsam eos\ntempora quo necessitatibus\ndolor quam autem quasi\nreiciendis et nam sapiente accusantium'),
 (2,
  1,
  'quo vero reiciendis velit similique earum',
  'Jayne_Kuhic@sydney.com',
  'est natus enim nihil est dolore omnis voluptatem numquam\net omnis occaecati quod ullam at\nvoluptatem error expedita pariatur\nnihil sint nostrum voluptatem reiciendis et'),
 (3,
  1,
  'odio adipisci rerum aut animi',
  'Nikita@garfield.biz',
  'quia molestiae reprehenderit quasi aspernatur\naut expedita occaecati aliquam eveniet laudantium\nomnis quibusdam delectus saepe quia accusamus maiores nam est\ncum et ducimus et vero voluptates excepturi deleniti ratione'),
 (4,
  1,
  'alias odio sit',
  'Lew@alysha.tv',
  'non et atque\noccaecati deserunt quas accusantium unde odit nobis qui voluptatem\nquia voluptas consequuntur itaque dolor\net qui rerum deleniti ut occaecati'),


## Step 5: Close the Connection
Always close the connection when done.

In [5]:
# Close the connection
cursor.close()
conn.close()


# SQL Exercises: Complex Joins and Aggregate Functions

Exercise 1: Count the Total Number of Posts per User

Write a query to count how many posts each user has created. This involves joining the users table with the posts table and using the COUNT() aggregate function.

In [6]:
import sqlite3

# Reconnect to the database
conn = sqlite3.connect(str(uuid.uuid4()) + '.db')  # Creates a new SQLite database
cursor = conn.cursor()

In [9]:
import sqlite3

# Reconnect to the database (use a fixed name to avoid creating a new empty DB)
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Create tables
create_users_table = '''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    username TEXT,
    email TEXT,
    address TEXT,
    phone TEXT,
    website TEXT,
    company TEXT
);
'''

create_posts_table = '''
CREATE TABLE IF NOT EXISTS posts (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    body TEXT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''

# Execute table creation
cursor.execute(create_users_table)
cursor.execute(create_posts_table)
conn.commit()
print("Tables created successfully!")

Tables created successfully!


In [10]:
import requests

# Function to fetch data from API
def fetch_data(endpoint):
    url = f'https://jsonplaceholder.typicode.com/{endpoint}'
    response = requests.get(url)
    return response.json()

# Fetch data
users_data = fetch_data('users')
posts_data = fetch_data('posts')

# Insert users data
def insert_users(data):
    for user in data:
        cursor.execute("""
        INSERT OR IGNORE INTO users (id, name, username, email, address, phone, website, company)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (user['id'], user['name'], user['username'], user['email'],
              str(user['address']), user['phone'], user['website'], str(user['company'])))
    conn.commit()

insert_users(users_data)

# Insert posts data
def insert_posts(data):
    for post in data:
        cursor.execute("""
        INSERT OR IGNORE INTO posts (id, user_id, title, body)
        VALUES (?, ?, ?, ?)
        """, (post['id'], post['userId'], post['title'], post['body']))
    conn.commit()

insert_posts(posts_data)

print("Data inserted successfully!")

Data inserted successfully!


In [11]:
query = """
SELECT users.id, users.name, COUNT(posts.id) AS post_count
FROM users
JOIN posts ON users.id = posts.user_id
GROUP BY users.id
ORDER BY post_count DESC;
"""
cursor.execute(query)
result = cursor.fetchall()
display(pd.DataFrame(result, columns=['User ID', 'Name', 'Total Posts']))


,User ID,Name,Total Posts
0,10,Clementina DuBuque,10
1,9,Glenna Reichert,10
2,8,Nicholas Runolfsdottir V,10
3,7,Kurtis Weissnat,10
4,6,Mrs. Dennis Schulist,10
5,5,Chelsey Dietrich,10
6,4,Patricia Lebsack,10
7,3,Clementine Bauch,10
8,2,Ervin Howell,10
9,1,Leanne Graham,10


Exercise 2: Retrieve Users with the Most Comments on Their Posts

Write a query to determine which users have received the most comments on their posts. This involves joining the users, posts, and comments tables and grouping by user.

In [14]:
# Create comments table if it doesn't exist
create_comments_table = '''
CREATE TABLE IF NOT EXISTS comments (
    id INTEGER PRIMARY KEY,
    post_id INTEGER,
    name TEXT,
    email TEXT,
    body TEXT,
    FOREIGN KEY (post_id) REFERENCES posts(id)
);
'''
cursor.execute(create_comments_table)
conn.commit()
print("Comments table created successfully!")

# Fetch and insert comments data
import requests

def fetch_data(endpoint):
    url = f'https://jsonplaceholder.typicode.com/{endpoint}'
    response = requests.get(url)
    return response.json()

comments_data = fetch_data('comments')

def insert_comments(data):
    for comment in data:
        cursor.execute("""
        INSERT OR IGNORE INTO comments (id, post_id, name, email, body)
        VALUES (?, ?, ?, ?, ?)
        """, (comment['id'], comment['postId'], comment['name'], comment['email'], comment['body']))
    conn.commit()

insert_comments(comments_data)
print("Comments data inserted successfully!")

Comments table created successfully!
Comments data inserted successfully!


In [15]:
query = """
SELECT users.id, users.name, COUNT(comments.id) AS total_comments
FROM users
JOIN posts ON users.id = posts.user_id
JOIN comments ON posts.id = comments.post_id
GROUP BY users.id
ORDER BY total_comments DESC;
"""
cursor.execute(query)
result = cursor.fetchall()
display(pd.DataFrame(result, columns=['User ID', 'Name', 'Total Comments']))


,User ID,Name,Total Comments
0,10,Clementina DuBuque,50
1,9,Glenna Reichert,50
2,8,Nicholas Runolfsdottir V,50
3,7,Kurtis Weissnat,50
4,6,Mrs. Dennis Schulist,50
5,5,Chelsey Dietrich,50
6,4,Patricia Lebsack,50
7,3,Clementine Bauch,50
8,2,Ervin Howell,50
9,1,Leanne Graham,50


Exercise 3: List the Top 3 Most Commented Posts

Write a query to find out which posts have received the most comments. Join the posts and comments tables and display the top 3 results.


In [ ]:
cursor.execute("#")
result = cursor.fetchall()
result

In [17]:
query = """
SELECT posts.id, posts.title, COUNT(comments.id) AS comment_count
FROM posts
JOIN comments ON posts.id = comments.post_id
GROUP BY posts.id
ORDER BY comment_count DESC
LIMIT 3;
"""
cursor.execute(query)
result = cursor.fetchall()
display(pd.DataFrame(result, columns=['Post ID', 'Title', 'Total Comments']))

,Post ID,Title,Total Comments
0,100,at nam consequatur ea labore ea harum,5
1,99,temporibus sit alias delectus eligendi possimu...,5
2,98,laboriosam dolor voluptates,5


Exercise 4: Find Albums that Have More than 10 Photos

Write a query to count how many photos are associated with each album and return only those albums that contain more than 10 photos.


In [ ]:
cursor.execute("#")
result = cursor.fetchall()
result

In [22]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Existing tables:", tables)

Existing tables: [('users',), ('posts',), ('comments',)]


In [24]:
create_albums_table = '''
CREATE TABLE IF NOT EXISTS albums (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''

create_photos_table = '''
CREATE TABLE IF NOT EXISTS photos (
    id INTEGER PRIMARY KEY,
    album_id INTEGER,
    title TEXT,
    url TEXT,
    thumbnail_url TEXT,
    FOREIGN KEY (album_id) REFERENCES albums(id)
);
'''

cursor.execute(create_albums_table)
cursor.execute(create_photos_table)
conn.commit()
print("Albums and Photos tables created successfully!")



Albums and Photos tables created successfully!


In [25]:
import requests

# Function to fetch data
def fetch_data(endpoint):
    url = f'https://jsonplaceholder.typicode.com/{endpoint}'
    response = requests.get(url)
    return response.json()

# Fetch albums and photos data
albums_data = fetch_data('albums')
photos_data = fetch_data('photos')

# Insert albums data
def insert_albums(data):
    for album in data:
        cursor.execute("""
        INSERT OR IGNORE INTO albums (id, user_id, title)
        VALUES (?, ?, ?)
        """, (album['id'], album['userId'], album['title']))
    conn.commit()

insert_albums(albums_data)

# Insert photos data
def insert_photos(data):
    for photo in data:
        cursor.execute("""
        INSERT OR IGNORE INTO photos (id, album_id, title, url, thumbnail_url)
        VALUES (?, ?, ?, ?, ?)
        """, (photo['id'], photo['albumId'], photo['title'], photo['url'], photo['thumbnailUrl']))
    conn.commit()

insert_photos(photos_data)

print("Albums and Photos data inserted successfully!")


Albums and Photos data inserted successfully!


In [26]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Existing tables:", tables)

Existing tables: [('users',), ('posts',), ('comments',), ('albums',), ('photos',)]


In [27]:
query = """
SELECT albums.id, albums.title, COUNT(photos.id) AS photo_count
FROM albums
JOIN photos ON albums.id = photos.album_id
GROUP BY albums.id
HAVING COUNT(photos.id) > 10
ORDER BY COUNT(photos.id) DESC;
"""
cursor.execute(query)
result = cursor.fetchall()
display(pd.DataFrame(result, columns=['Album ID', 'Title', 'Total Photos']))


,Album ID,Title,Total Photos
0,100,enim repellat iste,50
1,99,consectetur ut id impedit dolores sit ad ex aut,50
2,98,omnis quia possimus nesciunt deleniti assumend...,50
3,97,est quod aut,50
4,96,unde et ut molestiae est molestias voluptatem ...,50
...,...,...,...
95,5,eaque aut omnis a,50
96,4,non esse culpa molestiae omnis sed optio,50
97,3,omnis laborum odio,50
98,2,sunt qui excepturi placeat culpa,50


Exercise 5: Find Users with Incomplete Todos

Write a query to retrieve users who have at least one incomplete to-do item. Join the users and todos tables and use the WHERE clause to filter based on the completed field.

In [ ]:
cursor.execute("#")
result = cursor.fetchall()
result

In [29]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Existing tables:", tables)



Existing tables: [('users',), ('posts',), ('comments',), ('albums',), ('photos',)]


In [30]:
create_todos_table = '''
CREATE TABLE IF NOT EXISTS todos (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    completed BOOLEAN,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''
cursor.execute(create_todos_table)
conn.commit()
print("Todos table created successfully!")


Todos table created successfully!


In [31]:
import requests

# Fetch todos data
def fetch_data(endpoint):
    url = f'https://jsonplaceholder.typicode.com/{endpoint}'
    response = requests.get(url)
    return response.json()

todos_data = fetch_data('todos')

# Insert todos into the table
def insert_todos(data):
    for todo in data:
        cursor.execute("""
        INSERT OR IGNORE INTO todos (id, user_id, title, completed)
        VALUES (?, ?, ?, ?)
        """, (todo['id'], todo['userId'], todo['title'], todo['completed']))
    conn.commit()

insert_todos(todos_data)
print("Todos data inserted successfully!")


Todos data inserted successfully!


In [32]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Existing tables:", tables)


Existing tables: [('users',), ('posts',), ('comments',), ('albums',), ('photos',), ('todos',)]


In [33]:
query = """
SELECT users.id, users.name, COUNT(todos.id) AS incomplete_todos
FROM users
JOIN todos ON users.id = todos.user_id
WHERE todos.completed = 0
GROUP BY users.id
ORDER BY incomplete_todos DESC;
"""
cursor.execute(query)
result = cursor.fetchall()
display(pd.DataFrame(result, columns=['User ID', 'Name', 'Incomplete Todos']))


,User ID,Name,Incomplete Todos
0,6,Mrs. Dennis Schulist,14
1,4,Patricia Lebsack,14
2,3,Clementine Bauch,13
3,9,Glenna Reichert,12
4,2,Ervin Howell,12
5,7,Kurtis Weissnat,11
6,8,Nicholas Runolfsdottir V,9
7,1,Leanne Graham,9
8,10,Clementina DuBuque,8
9,5,Chelsey Dietrich,8


Exercise 6: Calculate the Average Number of Comments per Post

Write a query to calculate the average number of comments per post. This involves joining the posts and comments tables and using the AVG() function.


In [ ]:
cursor.execute("#")
result = cursor.fetchall()
result

In [34]:
query = """
SELECT AVG(comment_count) AS avg_comments_per_post
FROM (
    SELECT posts.id, COUNT(comments.id) AS comment_count
    FROM posts
    LEFT JOIN comments ON posts.id = comments.post_id
    GROUP BY posts.id
);
"""
cursor.execute(query)
result = cursor.fetchall()
display(pd.DataFrame(result, columns=['Average Comments Per Post']))


,Average Comments Per Post
0,5.0


Exercise 7: Retrieve Users Who Have Both Posts and Todos

Write a query to retrieve users who have created at least one post and one to-do item. You can use subqueries or a combination of JOIN and INTERSECT.

In [ ]:
cursor.execute("#")
result = cursor.fetchall()
result

NameError: name 'cursor' is not defined

In [ ]:
create_users_table = '''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    username TEXT,
    email TEXT,
    address TEXT,
    phone INTEGER,
    website TEXT,
    company TEXT,
    type integer foreign key;
    happy intger foreign key;

    0
    2
    4
    2
    9
);
'''

In [35]:
query = """
SELECT users.id, users.name
FROM users
WHERE users.id IN (SELECT DISTINCT user_id FROM posts)
AND users.id IN (SELECT DISTINCT user_id FROM todos);
"""
cursor.execute(query)
result = cursor.fetchall()
display(pd.DataFrame(result, columns=['User ID', 'Name']))


,User ID,Name
0,1,Leanne Graham
1,2,Ervin Howell
2,3,Clementine Bauch
3,4,Patricia Lebsack
4,5,Chelsey Dietrich
5,6,Mrs. Dennis Schulist
6,7,Kurtis Weissnat
7,8,Nicholas Runolfsdottir V
8,9,Glenna Reichert
9,10,Clementina DuBuque


In [36]:
cursor.close()
conn.close()
print("Database connection closed successfully!")

Database connection closed successfully!
